In [0]:
df = spark.read.option('header', 'true')\
        .option('inferSchema', 'true')\
        .csv('/Volumes/databricks_simulated_retail_customer_data/v01/source_files/sales.csv')

display(df.limit(10))

In [0]:
df.printSchema()

In [0]:
df = spark.read\
        .option('header' , 'true')\
        .option('quote', '"')\
        .option('escape', '"')\
        .option('multiline','true')\
        .csv('/Volumes/databricks_simulated_retail_customer_data/v01/source_files/sales.csv')

display(df.limit(10))

In [0]:
df.select('customer_id').count()

In [0]:
df.select('product').limit(10).display()

df.select('product').count()

defining schema for only json column 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [0]:
product_schema = StructType([
    StructField("curr", StringType(), True),
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("qty", IntegerType(), True),
    StructField("unit", IntegerType(), True)
])

# struct type 

In [0]:
df_parsed = df.withColumn('product_parsed',
                          from_json(col("product") , product_schema)
                          )

display(df_parsed.limit(10))

In [0]:
df_flat = (
            df_parsed\
              .withColumn('currency' , col("product_parsed.curr"))\
              .withColumn('product_id' , col("product_parsed.id"))\
              .withColumn('name' , col("product_parsed.name"))\
              .withColumn('product_price' , col("product_parsed.price"))\
              .withColumn('product_qty' , col("product_parsed.qty"))\
              .withColumn('product_unit' , col("product_parsed.unit"))\

              .drop("product", "product_parsed")
                          
)

display(df_flat.limit(10))
df_flat.printSchema()


In [0]:

df = df_flat \
    .withColumn("customer_id", col("customer_id").cast("int")) \
    .withColumn("order_date", col("order_date").cast("date")) \
    .withColumn("total_price", col("total_price").cast("double"))

df.printSchema()

In [0]:
display(df.limit(20))

In [0]:
df = df.withColumn(
    "customer_name",
    initcap(regexp_replace("customer_name", ",", ""))
)
display(df.limit(5))

In [0]:
df.write.mode("overwrite").saveAsTable('analytics_star_schema.deltashare.fact_sales_0')

In [0]:
%sql
select * from analytics_star_schema.deltashare.fact_sales_0 limit 15

In [0]:
%sql
SELECT
  customer_id,
  COUNT(*) AS cnt
FROM
  analytics_star_schema.deltashare.fact_sales_0
GROUP BY
  customer_id
HAVING
  COUNT(*) > 1
ORDER BY
  cnt DESC

In [0]:
%sql
select *, count(*) over() as count
from analytics_star_schema.deltashare.fact_sales_0 
where customer_id = 24633905

In [0]:
df.select("customer_id").count()

In [0]:
distinct_customer_id_count = df.select(
    countDistinct("customer_id").alias("distinct_customer_id_count")
)

display(distinct_customer_id_count)

In [0]:
%sql
select * from analytics_star_schema.deltashare.fact_sales_0 limit 5

In [0]:
df_temp = df.dropDuplicates(["customer_id"]) 

display(df_temp)

In [0]:
df_temp.count()

In [0]:
df = spark.read.table('analytics_star_schema.deltashare.fact_sales_0') 

#display(df.limit(10))

df.count()

In [0]:
df_prod_id = df.select(countDistinct("product_id"))

display(df_prod_id)